<a href="https://colab.research.google.com/github/kskaran94/VisualSimilarity/blob/master/Image_Similarity_Baseline_Celeb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Running script on TF 2.0

In [1]:
try:
  %tensorflow_version 2.x # enable TF 2.x in Colab
except Exception:
  pass

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `2.x # enable TF 2.x in Colab`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


### Libraries imported

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from scipy import spatial
from os import listdir
import IPython.display as display
import matplotlib.pyplot as plt
from random import randrange

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


### Download VGG

In [4]:
model = VGG16(weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 2s 0us/step


In [0]:
cp gdrive/My\ Drive/celeb.zip .

### unzip file

In [0]:
%%capture
!unzip celeb.zip -d celeb

### Image to Embedding

In [0]:
base_path = '/content/celeb'

In [8]:
import pathlib

train_images_folder = pathlib.Path(base_path+'/train')
test_images_folder = pathlib.Path(base_path+'/test')

print(train_images_folder)

/content/celeb/train


In [0]:
train_image_paths = list(train_images_folder.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]

test_image_paths = list(test_images_folder.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]

In [10]:
print(len(train_image_paths))
print(len(test_image_paths))

1144
286


In [11]:
train_image_paths[:10]

['/content/celeb/train/David_Nalbandian/David_Nalbandian_0012.jpg',
 '/content/celeb/train/David_Nalbandian/David_Nalbandian_0005.jpg',
 '/content/celeb/train/David_Nalbandian/David_Nalbandian_0008.jpg',
 '/content/celeb/train/David_Nalbandian/David_Nalbandian_0001.jpg',
 '/content/celeb/train/David_Nalbandian/David_Nalbandian_0009.jpg',
 '/content/celeb/train/David_Nalbandian/David_Nalbandian_0006.jpg',
 '/content/celeb/train/David_Nalbandian/David_Nalbandian_0002.jpg',
 '/content/celeb/train/David_Nalbandian/David_Nalbandian_0014.jpg',
 '/content/celeb/train/Kim_Clijsters/Kim_Clijsters_0010.jpg',
 '/content/celeb/train/Kim_Clijsters/Kim_Clijsters_0004.jpg']

In [0]:
train_labels = [pathlib.Path(path).parent.name for path in train_image_paths] #path is str
test_labels = [pathlib.Path(path).parent.name for path in test_image_paths]

In [13]:
train_labels[:10]

['David_Nalbandian',
 'David_Nalbandian',
 'David_Nalbandian',
 'David_Nalbandian',
 'David_Nalbandian',
 'David_Nalbandian',
 'David_Nalbandian',
 'David_Nalbandian',
 'Kim_Clijsters',
 'Kim_Clijsters']

In [14]:
test_labels[:10]

['David_Nalbandian',
 'David_Nalbandian',
 'Kim_Clijsters',
 'Kim_Clijsters',
 'Jackie_Chan',
 'Jackie_Chan',
 'Jiang_Zemin',
 'Jiang_Zemin',
 'Wen_Jiabao',
 'Wen_Jiabao']

In [0]:
def get_embeddings(images_path):
  feature_list = []
  
  for img in images_path:
    img_1 = image.load_img(img, target_size=(224, 224)) 
    x = image.img_to_array(img_1)
    #print(x.shape)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features_1 = model.predict(x).ravel()
    feature_list.append(features_1)

  return np.array(feature_list)

In [0]:
train_embeddings = get_embeddings(train_image_paths)

In [17]:
train_embeddings.shape

(1144, 25088)

In [0]:
test_embeddings = get_embeddings(test_image_paths)

In [19]:
test_embeddings.shape

(286, 25088)

# Evaluation on the test set

In [0]:
correct_identified = 0

for index, test_img_embedding in enumerate(test_embeddings):
  diff = (train_embeddings - test_img_embedding)**2
  dist = np.sum(diff, axis=1)
  nearest_neighbors = np.argsort(dist)[:10]
  
  if test_labels[index] in np.array(train_labels)[nearest_neighbors[:10]]:
    #print(test_labels[index], train_full_labels[nearest_neighbors[0]])
    correct_identified += 1


In [23]:
print("Accuracy is ", correct_identified / test_embeddings.shape[0])


Accuracy is  0.4020979020979021
